<a href="https://colab.research.google.com/github/FrodoBaggins87/NLP_Projects/blob/main/Poetry_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Make imports

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense
print(f"Tensorflow version:{tf.__version__}")

Tensorflow version:2.17.0


##Downloading Data

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2024-08-06 16:19:40--  https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.207, 142.250.152.207, 172.217.214.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.001s  

2024-08-06 16:19:40 (68.9 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



Viewing the data

In [3]:
!cat /tmp/irish-lyrics-eof.txt


Come all ye maidens young and fair
And you that are blooming in your prime
Always beware and keep your garden fair
Let no man steal away your thyme
For thyme it is a precious thing
And thyme brings all things to my mind
nlyme with all its flavours, along with all its joys
Thyme, brings all things to my mind
Once I and a bunch of thyme
i thought it never would decay
Then came a lusty sailor
Who chanced to pass my way
And stole my bunch of thyme away
The sailor gave to me a rose
A rose that never would decay
He gave it to me to keep me reminded
Of when he stole my thyme away
Sleep, my child, and peace attend thee
All through the night
Guardian angels God will send thee
Soft the drowsy hours are creeping
Hill and dale in slumber sleeping
I my loving vigil keeping
While the moon her watch is keeping
While the weary world is sleeping
Oer thy spirit gently stealing
Visions of delight revealing
Breathes a pure and holy feeling
Though I roam a minstrel lonely
My true harp shall praise sing onl

Assigning data to a variable name

In [4]:
# Open the file and read its contents into the variable 'data'
with open('/tmp/irish-lyrics-eof.txt', 'r') as file:
    data = file.read()

# Print the contents of the variable 'data' to verify
print(data)


Come all ye maidens young and fair
And you that are blooming in your prime
Always beware and keep your garden fair
Let no man steal away your thyme
For thyme it is a precious thing
And thyme brings all things to my mind
nlyme with all its flavours, along with all its joys
Thyme, brings all things to my mind
Once I and a bunch of thyme
i thought it never would decay
Then came a lusty sailor
Who chanced to pass my way
And stole my bunch of thyme away
The sailor gave to me a rose
A rose that never would decay
He gave it to me to keep me reminded
Of when he stole my thyme away
Sleep, my child, and peace attend thee
All through the night
Guardian angels God will send thee
Soft the drowsy hours are creeping
Hill and dale in slumber sleeping
I my loving vigil keeping
While the moon her watch is keeping
While the weary world is sleeping
Oer thy spirit gently stealing
Visions of delight revealing
Breathes a pure and holy feeling
Though I roam a minstrel lonely
My true harp shall praise sing onl

In [5]:
# Print the contents of the variable 'data' using repr to show \n characters
print(repr(data))


'Come all ye maidens young and fair\nAnd you that are blooming in your prime\nAlways beware and keep your garden fair\nLet no man steal away your thyme\nFor thyme it is a precious thing\nAnd thyme brings all things to my mind\nnlyme with all its flavours, along with all its joys\nThyme, brings all things to my mind\nOnce I and a bunch of thyme\ni thought it never would decay\nThen came a lusty sailor\nWho chanced to pass my way\nAnd stole my bunch of thyme away\nThe sailor gave to me a rose\nA rose that never would decay\nHe gave it to me to keep me reminded\nOf when he stole my thyme away\nSleep, my child, and peace attend thee\nAll through the night\nGuardian angels God will send thee\nSoft the drowsy hours are creeping\nHill and dale in slumber sleeping\nI my loving vigil keeping\nWhile the moon her watch is keeping\nWhile the weary world is sleeping\nOer thy spirit gently stealing\nVisions of delight revealing\nBreathes a pure and holy feeling\nThough I roam a minstrel lonely\nMy t

##Tokenizing

In [6]:
#make tokenizer object
tokenizer=Tokenizer(oov_token="<OOV>")

#make corpus of data
corpus=data.lower().split("\n")

#fit tokenizer on corpus
tokenizer.fit_on_texts(corpus)

#fixing total number of words
total_words=len(tokenizer.word_index)+1

print(f"Total words:{total_words}")


Total words:2691


In [7]:
print(tokenizer.word_index)

{'<OOV>': 1, 'the': 2, 'and': 3, 'i': 4, 'to': 5, 'a': 6, 'of': 7, 'my': 8, 'in': 9, 'me': 10, 'for': 11, 'you': 12, 'all': 13, 'was': 14, 'she': 15, 'that': 16, 'on': 17, 'with': 18, 'her': 19, 'but': 20, 'as': 21, 'when': 22, 'love': 23, 'is': 24, 'your': 25, 'it': 26, 'will': 27, 'from': 28, 'by': 29, 'they': 30, 'be': 31, 'are': 32, 'so': 33, 'he': 34, 'old': 35, 'no': 36, 'oh': 37, 'ill': 38, 'at': 39, 'one': 40, 'his': 41, 'there': 42, 'were': 43, 'heart': 44, 'down': 45, 'now': 46, 'we': 47, 'where': 48, 'young': 49, 'never': 50, 'go': 51, 'come': 52, 'then': 53, 'did': 54, 'not': 55, 'said': 56, 'away': 57, 'their': 58, 'sweet': 59, 'them': 60, 'green': 61, 'if': 62, 'take': 63, 'our': 64, 'like': 65, 'night': 66, 'day': 67, 'o': 68, 'out': 69, 'fair': 70, 'this': 71, 'town': 72, 'have': 73, 'can': 74, 'true': 75, 'its': 76, 'thou': 77, 'see': 78, 'dear': 79, 'more': 80, 'theres': 81, 'or': 82, 'had': 83, 'would': 84, 'over': 85, 'hear': 86, 'up': 87, 'ive': 88, 'through': 89, 

In [8]:
for line in corpus[:10]:
  print(line)

come all ye maidens young and fair
and you that are blooming in your prime
always beware and keep your garden fair
let no man steal away your thyme
for thyme it is a precious thing
and thyme brings all things to my mind
nlyme with all its flavours, along with all its joys
thyme, brings all things to my mind
once i and a bunch of thyme
i thought it never would decay


##Create input sequences for training

In [9]:
tokenizer.texts_to_sequences("my name is")

[[1], [1], [], [1790], [6], [1], [1], [], [4], [1]]

In [10]:
tokenizer.texts_to_sequences(["my name is"])

[[8, 450, 24]]

In [11]:
token_list=tokenizer.texts_to_sequences(["name"])
print(token_list)
print(tokenizer.word_index['name'])

[[450]]
450


In [12]:
corpus[1]

'and you that are blooming in your prime'

In [13]:
#make an empty list for storing input sequences
input_sequences=[]

#loop through each line in corpus
for line in corpus:
  try:
    #convert each line to sequence
    token_list=tokenizer.texts_to_sequences([line])[0] #as can be seen in above cell output texts_to_sequences method gives a list inside a list, thats why 0th element is listed here
    for i in range(1,len(token_list)):
      #make n gram sequences for each line converted to sequence
      n_gram_sequences=token_list[:i+1]

      #append each n gram sequence in input_sequence
      input_sequences.append(n_gram_sequences)
  except Exception as e:
    pass


In [14]:
input_sequences

[[52, 13],
 [52, 13, 97],
 [52, 13, 97, 1218],
 [52, 13, 97, 1218, 49],
 [52, 13, 97, 1218, 49, 3],
 [52, 13, 97, 1218, 49, 3, 70],
 [3, 12],
 [3, 12, 16],
 [3, 12, 16, 32],
 [3, 12, 16, 32, 362],
 [3, 12, 16, 32, 362, 9],
 [3, 12, 16, 32, 362, 9, 25],
 [3, 12, 16, 32, 362, 9, 25, 1219],
 [273, 799],
 [273, 799, 3],
 [273, 799, 3, 205],
 [273, 799, 3, 205, 25],
 [273, 799, 3, 205, 25, 580],
 [273, 799, 3, 205, 25, 580, 70],
 [119, 36],
 [119, 36, 120],
 [119, 36, 120, 800],
 [119, 36, 120, 800, 57],
 [119, 36, 120, 800, 57, 25],
 [119, 36, 120, 800, 57, 25, 185],
 [11, 185],
 [11, 185, 26],
 [11, 185, 26, 24],
 [11, 185, 26, 24, 6],
 [11, 185, 26, 24, 6, 581],
 [11, 185, 26, 24, 6, 581, 457],
 [3, 185],
 [3, 185, 801],
 [3, 185, 801, 13],
 [3, 185, 801, 13, 802],
 [3, 185, 801, 13, 802, 5],
 [3, 185, 801, 13, 802, 5, 8],
 [3, 185, 801, 13, 802, 5, 8, 236],
 [1220, 18],
 [1220, 18, 13],
 [1220, 18, 13, 76],
 [1220, 18, 13, 76, 1221],
 [1220, 18, 13, 76, 1221, 237],
 [1220, 18, 13, 76, 1

In [15]:
len(input_sequences)

12038

In [16]:
max_sequence_length=max([len(x) for x in input_sequences])
max_sequence_length

16

##Padding Sequences

In [17]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_length,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,   52,   13],
       [   0,    0,    0, ...,   52,   13,   97],
       [   0,    0,    0, ...,   13,   97, 1218],
       ...,
       [   0,    0,    0, ...,    0,   48,  106],
       [   0,    0,    0, ...,   48,  106,  139],
       [   0,    0,    0, ...,  106,  139,  185]], dtype=int32)

##Making input and output lists

In [23]:
xs=input_sequences[:,:-1]
y=input_sequences[:,-1]

In [24]:
len(xs[0])

15

Doing one hot encoding on y

In [25]:
ys=tf.keras.utils.to_categorical(y,num_classes=total_words)
print(len(ys))
print(ys.shape)

12038
(12038, 2691)


##Making the neural network

In [26]:
#1st layer Embedding
#2nd layer Bidirectional LSTM
#3rd layer Dense
#Optimizer Adam
#loss function categorical cross entropy
#metric= accuracy
model=Sequential()
model.add(Embedding(total_words,240,input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(xs,ys,epochs=10,verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 43s 102ms/step - accuracy: 0.0587 - loss: 6.9395
Epoch 2/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.0775 - loss: 6.1716
Epoch 3/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 35s 94ms/step - accuracy: 0.0931 - loss: 5.8361
Epoch 4/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 34s 91ms/step - accuracy: 0.1224 - loss: 5.4043
Epoch 5/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 42s 94ms/step - accuracy: 0.1361 - loss: 5.0355
Epoch 6/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 43s 113ms/step - accuracy: 0.1662 - loss: 4.6638
Epoch 7/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 81s 111ms/step - accuracy: 0.2044 - loss: 4.2866
Epoch 8/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 81s 110ms/step - accuracy: 0.2414 - loss: 3.9672
Epoch 9/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 81s 109ms/step - accuracy: 0.2863 - loss: 3.6304
Epoch 10/10
377/377 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - accuracy: 0.3339 - loss: 3.3408


##Generate text

In [28]:
seed_text="I you me they"
next_words=20

for _ in range(next_words):
  #make token list
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  #pad the token list
  token_list=pad_sequences([token_list],maxlen=max_sequence_length-1,padding='pre')
  #predict the next word
  predicted=model.predict(token_list,verbose=0)
  #find index of word
  predicted=np.argmax(predicted,axis=-1)[0]
  output_word=""
  for word,index in tokenizer.word_index.items():
    if index==predicted:
      output_word=word
      break
  seed_text+=" "+output_word
#print generated text after seed text
print(seed_text)




I you me they love me kathleen love me now dhu grow by side by side and bridge bridge pots small weirs bragh malone
